# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [19]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing, libmpMuelMat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# set the parameters to run the script
directories = [r'F:\test_dev']
calib_directory = r'F:\polarimetry\calibration'

In [21]:
# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
# set the parameters to be used for the line visualisation with the parameter parameter_set
batch_processing.batch_process(directories, calib_directory, run_all = True, parameter_set = 'PD')

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:31<00:00, 31.62s/it]
